In [1]:
import csv
import numpy as np
import pandas as pd
from dateutil import parser as dt_parse
from datetime import datetime

# from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC

from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor

# from sklearn.neural_network import MLPClassifier

## Age category:
# [0-12): C1
# [12, 25): C2
# [25, 60): C3
# 60+: C4

data = pd.read_csv("/home/krullun/Documents/coding/JPMorgan/Problem_1/train.csv")
test_data = pd.read_csv("/home/krullun/Documents/coding/JPMorgan/Problem_1/test.csv")

In [2]:
print(data.Name)

0        Dr. FG61 FS88
1        Dr. FG35 FS74
2        Dr. MG45 MS99
3        Mr. MG93 MS13
4       Miss FG28 FS14
5       Miss FG66 FS95
6        Mr. MG71 MS64
7         Dr. MG32 MS6
8        Mr. MG69 MS30
9        Mr. MG14 MS53
10       Dr. MG24 MS32
11       Dr. FG67 FS83
12       Mr. MG98 MS64
13      Miss FG67 FS23
14      Miss FG20 FS26
15      Miss FG87 FS90
16       Dr. FG64 FS65
17       Mr. MG89 MS87
18       Dr. FG49 FS68
19       Mr. MG21 MS89
20        Dr. MG6 MS82
21       Dr. MG65 MS53
22      Mrs. FG53 FS26
23       Dr. MG49 MS59
24        Dr. FG9 FS29
25       Mr. MG20 MS65
26       Mr. MG79 MS47
27       Dr. FG83 FS97
28      Mrs. FG70 FS73
29       Dr. MG50 MS41
             ...      
7470    Miss FG13 FS51
7471    Mrs. FG67 FS29
7472      Mr. MG0 MS43
7473    Mrs. FG16 FS60
7474     Mr. MG62 MS64
7475     Dr. MG33 MS23
7476     Mr. MG70 MS95
7477     Dr. FG27 FS57
7478     Dr. MG38 MS27
7479      Mr. MG52 MS3
7480    Miss FG97 FS33
7481    Miss FG97 FS31
7482    Mis

In [21]:
city = data.From.unique().tolist()
flight_class = data.Class.unique().tolist() 

current_datetime = datetime.now()
# flight_date = data['Flight Date']
# flight_time = data['Flight Time']
# booking_date = data['Booking Date']
# for k,i in flight_date.items():
# 	print(k)
# 	print(i, booking_date[k])
# 	print((dt_parse.parse(i) - dt_parse.parse(booking_date[k])).days)
# 	if k > 10:
# 		break


mean = np.zeros((len(city), len(city)))
std_dev = np.zeros((len(city), len(city)))
minimum = np.zeros((len(city), len(city)))
maximum = np.zeros((len(city), len(city)))
for source in city:
	for destination in city:
		fares = data.Fare[(data['From'] == source) & (data['To'] == destination)]
		mean[city.index(source)][city.index(destination)] = fares.mean()
		std_dev[city.index(source)][city.index(destination)] = fares.std()
		minimum[city.index(source)][city.index(destination)] = fares.min()
		maximum[city.index(source)][city.index(destination)] = fares.max()
# print(mean)
# print(std_dev)

X_train = []
Y_train = []

for ind,row in data.iterrows():
	city_vec = np.zeros(len(city))
	city_vec[city.index(row['From'])] = 2
	city_vec[city.index(row['To'])] = 1
	# print(city_vec)

	age_vec = np.zeros(4)
	age = ((dt_parse.parse(row['Booking Date']) - dt_parse.parse(row['Date of Birth'])).days)/365
	if age < 12:
		age_vec[0] = 1
	elif age < 25:
		age_vec[1] = 1
	elif age < 50:
		age_vec[2] = 1
	else:
		age_vec[3] = 1
	# print(age_vec)

	duration = (dt_parse.parse(row['Flight Date']) - dt_parse.parse(row['Booking Date'])).days

	weekday_vec = np.zeros(7)
	weekday_ind = ((current_datetime - dt_parse.parse(row['Flight Date'])).days)%7
	weekday_vec[weekday_ind] = 1
	# print(duration*24*60)

	time = dt_parse.parse(row['Flight Time'])
	flight_time = time.hour*60+time.minute+time.second
	# print(time)
	# print(flight_time)

	flight_class_vec = np.zeros(len(flight_class))
	flight_class_vec[flight_class.index(row['Class'])] = 1

	# mean_ = mean[city.index(row['From'])][city.index(row['To'])]
	# std_ = std_dev[city.index(row['From'])][city.index(row['To'])]
	# norm_fare = (row['Fare'] - mean_)/(8*std_)
	min_ = minimum[city.index(row['From'])][city.index(row['To'])]
	max_ = maximum[city.index(row['From'])][city.index(row['To'])]
	norm_fare = (row['Fare'])/(max_)
	# print(norm_fare)

	feature_vec = np.concatenate((city_vec, flight_class_vec))
	feature_vec = np.concatenate((np.concatenate((feature_vec, weekday_vec)), np.asarray([duration, flight_time, ])))
	# print(feature_vec)
	# print(norm_fare)
	X_train.append(feature_vec)
	Y_train.append(norm_fare)
X_train_mat = np.asarray(X_train)
Y_train_mat = np.asarray(Y_train)	


X_test = []
for ind,row in test_data.iterrows():
	city_vec = np.zeros(len(city))
	city_vec[city.index(row['From'])] = 2
	city_vec[city.index(row['To'])] = 1
	# print(city_vec)

	age_vec = np.zeros(4)
	age = ((dt_parse.parse(row['Booking Date']) - dt_parse.parse(row['Date of Birth'])).days)/365
	if age < 12:
		age_vec[0] = 1
	elif age < 25:
		age_vec[1] = 1
	elif age < 50:
		age_vec[2] = 1
	else:
		age_vec[3] = 1
	# print(age_vec)

	duration = (dt_parse.parse(row['Flight Date']) - dt_parse.parse(row['Booking Date'])).days

	weekday_vec = np.zeros(7)
	weekday_ind = ((current_datetime - dt_parse.parse(row['Flight Date'])).days)%7
	weekday_vec[weekday_ind] = 1
	# print(duration*24*60)

	time = dt_parse.parse(row['Flight Time'])
	flight_time = time.hour*60+time.minute+time.second
	# print(time)
	# print(flight_time)

	flight_class_vec = np.zeros(len(flight_class))
	flight_class_vec[flight_class.index(row['Class'])] = 1

# 	feature_vec = np.concatenate((age_vec, np.concatenate((city_vec, flight_class_vec))))
	feature_vec = np.concatenate((city_vec, flight_class_vec))
	feature_vec = np.concatenate((np.concatenate((feature_vec, weekday_vec)), np.asarray([duration, flight_time])))
	# print(feature_vec)
	# print(norm_fare)
	X_test.append(feature_vec)
X_test_mat = np.asarray(X_test)


# print(len(X_train))
# print(len(Y_train))
# print(max(Y_train), min(Y_train))

# clf=LogisticRegression()
# clf = SVR()
# clf = KernelRidge(alpha=1.0)
clf = MLPRegressor(hidden_layer_sizes=(100, 50, 5))  
# 5, 50, 10, then 50, 200, 20 then 50, 100, 10 with [age_vec, city_vec, duration, flight_time]
## 128, 16 then 64, 16 then 10, 50, 20 with [age_vec, city_vec, flight_claas, weekday_vec, duration, fligt_time]
## 16, 128, 16, 16, 8,
split_ind = int(0.8*len(X_train))
clf.fit(X_train_mat[:split_ind],Y_train_mat[:split_ind])
Y_predict = clf.predict(X_train_mat[split_ind+1:])
y = []
for i in range(len(Y_predict)):
	if Y_train_mat[split_ind+1+i]==0:
		continue
	y.append(abs(Y_predict[i] - Y_train_mat[split_ind+1+i]/Y_train_mat[split_ind+1+i]))

# y = [abs(Y_predict[i] - Y_train_mat[split_ind+1+i]/Y_train_mat[split_ind+1+i]) for i in range(len(Y_predict))] 
# y = abs(Y_predict - Y_train_mat[split_ind+1:])/Y_train_mat[split_ind+1:] 
print('LogisticRegression error_score: ', np.mean(y))


Y = []
Y_test_predict = clf.predict(X_test_mat)
# i = 0
for ind,row in test_data.iterrows():
	min_ = minimum[city.index(row['From'])][city.index(row['To'])]
	max_ = maximum[city.index(row['From'])][city.index(row['To'])]
	Y.append(Y_test_predict[ind]*(max_))

with open('/home/krullun/Documents/coding/JPMorgan/Problem_1/out.txt', 'w') as f:
#     f.write(Y)
    for i in Y:
        f.write("%f\n" % i)

f.close()
print(Y)

LogisticRegression error_score:  0.7149799251983268


TypeError: write() argument must be str, not list